In [3]:
import numpy as np
import pandas as pd

In [4]:

def arbre_recomb(n,b):
    l=2*b*n+1
    S=np.zeros((l,n+1))
    for i in range(l):
        S[i,n]= b*n-i
    for j in range(1,n+1):
        for i in  range(l):
            if i>=b*j and i<=b*(2*n-j):
                S[i,n-j]=b*n-i
    return S

In [5]:
def proba_up_middle_down(List_sigma,List_a,Liste_saut,sigma_bar,T,n):
    proba=np.zeros((3,len(List_sigma)))
    h=T/n
    for i in range(len(List_sigma)):
        proba[0,i]=(List_sigma[i]**2+List_a[i]*Liste_saut[i]*sigma_bar*np.sqrt(h)+h*List_a[i]**2)/(2*(Liste_saut[i]*sigma_bar)**2)
        proba[1,i]=(List_sigma[i]**2-List_a[i]*Liste_saut[i]*sigma_bar*np.sqrt(h)+h*List_a[i]**2)/(2*(Liste_saut[i]*sigma_bar)**2)
        proba[2,i]=1-(proba[0,i]+proba[1,i])
    return proba

In [7]:
arbre=arbre_recomb(1000,2)

In [8]:
def prix_call_europeen(K,S_o,n,b,sigma_bar,T,List_sigma,List_a,Liste_saut,List_r,P):
    proba_umd=proba_up_middle_down(List_sigma,List_a,Liste_saut,sigma_bar,T,n)
    l=2*b*n+1
    h=T/n
    table_prix=np.zeros((l,n+1),dtype=object)

    for i in range(l):
        table_prix[i,n]={reg:max(-K+S_o*np.exp(arbre[i,n]*sigma_bar*np.sqrt(h)),0) for reg in range(len(List_sigma))}
    for j in range(1,n+1):
        for i in range(l):
            if i>=b*j and i<=b*(2*n-j):
                table_prix[i,n-j]={reg:0 for reg in range(len(List_sigma))}
                for etat in range(len(List_sigma)):
                    r=List_r[etat]
                    saut=Liste_saut[etat]
                    for k in range(len(List_sigma)):
                        table_prix[i,n-j][etat] += P[etat,k]*(table_prix[i-saut,n-j+1][k]*proba_umd[0,etat]+table_prix[i+saut,n-j+1][k]*proba_umd[1,etat]+table_prix[i,n-j+1][k]*proba_umd[2,etat])
                    table_prix[i,n-j][etat]=np.exp(-r*h)*table_prix[i,n-j][etat]
        
    return table_prix

In [9]:
def transition_matrix(Q,h):
    nbr=Q.shape[0]
    tr_matrix=np.zeros((nbr,nbr))
    for i in range(nbr):
        for j in range(nbr):
            if i==j:
                tr_matrix[i,j]=np.exp(Q[i,j]*h)
            else:
                tr_matrix[i,j]=-(1-np.exp(Q[i,i]*h))*Q[i,j]/Q[i,i]
    return tr_matrix

In [10]:
Q=np.array([[-0.5,0.5],[0.5,-0.5]])
P=transition_matrix(Q,1/1000)

In [11]:
P

array([[9.99500125e-01, 4.99875021e-04],
       [4.99875021e-04, 9.99500125e-01]])

In [12]:
L1=[0.15,0.25]
L2=[0.05-0.5*0.15**2,0.05-0.5*0.25**2]
L3=[1,2]
L4=[0.05,0.05]

In [13]:
proba_up_middle_down(L1,L2,L3,0.2,1,1000)

array([[0.28433223, 0.19605476],
       [0.27820531, 0.19457244],
       [0.43746246, 0.6093728 ]])

In [15]:
A1=prix_call_europeen(100,94,1000,2,0.2,1,L1,L2,L3,L4,P)
A2=prix_call_europeen(100,96,1000,2,0.2,1,L1,L2,L3,L4,P)
A3=prix_call_europeen(100,98,1000,2,0.2,1,L1,L2,L3,L4,P)
A4=prix_call_europeen(100,100,1000,2,0.2,1,L1,L2,L3,L4,P)
A5=prix_call_europeen(100,102,1000,2,0.2,1,L1,L2,L3,L4,P)
A6=prix_call_europeen(100,104,1000,2,0.2,1,L1,L2,L3,L4,P)
A7=prix_call_europeen(100,106,1000,2,0.2,1,L1,L2,L3,L4,P)

In [16]:
A = [A1, A2, A3, A4, A5, A6, A7]

In [17]:
Df = pd.DataFrame({'S0':[i for i in range(94,107,2)], 'Tree':[B[:,0][2000] for B in A]})
Df

,S0,Tree
0,94,"{0: 5.86121169713768, 1: 8.228404386030418}"
1,96,"{0: 6.923020326921834, 1: 9.317382007445124}"
2,98,"{0: 8.083383992632376, 1: 10.477573544944184}"
3,100,"{0: 9.338299622468098, 1: 11.704247134436248}"
4,102,"{0: 10.683620427004527, 1: 13.000574946330795}"
5,104,"{0: 12.111348355527737, 1: 14.356138588476941}"
6,106,"{0: 13.61462223374124, 1: 15.772456149244976}"


In [ ]:
def prix_put_europeen(K,S_o,n,b,sigma_bar,T,List_sigma,List_a,Liste_saut,List_r,P):
    proba_umd=proba_up_middle_down(List_sigma,List_a,Liste_saut,sigma_bar,T,n)
    l=2*b*n+1
    h=T/n
    table_prix=np.zeros((l,n+1),dtype=object)
    for i in range(l):
        table_prix[i,n]={reg:max(K-S_o*np.exp(arbre[i,n]*sigma_bar*np.sqrt(h)),0) for reg in range(len(List_sigma))}
    for j in range(1,n+1):
        for i in range(l):
            if i>=b*j and i<=b*(2*n-j):
                table_prix[i,n-j]={reg:0 for reg in range(len(List_sigma))}
                for etat in range(len(List_sigma)):
                    r=List_r[etat]
                    saut=Liste_saut[etat]
                    for k in range(len(List_sigma)):
                        table_prix[i,n-j][etat] += P[etat,k]*(table_prix[i-saut,n-j+1][k]*proba_umd[0,etat]+table_prix[i+saut,n-j+1][k]*proba_umd[1,etat]+table_prix[i,n-j+1][k]*proba_umd[2,etat])
                    table_prix[i,n-j][etat]=np.exp(-r*h)*table_prix[i,n-j][etat]
        
    return table_prix

In [ ]:
def prix_call_americain(K,S_o,n,b,sigma_bar,T,List_sigma,List_a,Liste_saut,List_r,P):
    proba_umd=proba_up_middle_down(List_sigma,List_a,Liste_saut,sigma_bar,T,n)
    l=2*b*n+1
    h=T/n
    table_prix=np.zeros((l,n+1),dtype=object)
    for i in range(l):
        table_prix[i,n]={reg:max(-K+S_o*np.exp(arbre[i,n]*sigma_bar*np.sqrt(h)),0) for reg in range(len(List_sigma))}
    for j in range(1,n+1):
        for i in range(l):
            if i>=b*j and i<=b*(2*n-j):
                table_prix[i,n-j]={reg:0 for reg in range(len(List_sigma))}
                for etat in range(len(List_sigma)):
                    r=List_r[etat]
                    saut=Liste_saut[etat]
                    for k in range(len(List_sigma)):
                        table_prix[i,n-j][etat] += P[etat,k]*(table_prix[i-saut,n-j+1][k]*proba_umd[0,etat]+table_prix[i+saut,n-j+1][k]*proba_umd[1,etat]+table_prix[i,n-j+1][k]*proba_umd[2,etat])
                    table_prix[i,n-j][etat]=np.exp(-r*h)*table_prix[i,n-j][etat]
                    table_prix[i,n-j][etat]=max(table_prix[i,n-j][etat],-K+S_o*np.exp(arbre[i,n-j]*sigma_bar*np.sqrt(h)))
        
    return table_prix

In [ ]:
def prix_put_americain(K,S_o,n,b,sigma_bar,T,List_sigma,List_a,Liste_saut,List_r,P):
    proba_umd=proba_up_middle_down(List_sigma,List_a,Liste_saut,sigma_bar,T,n)
    l=2*b*n+1
    h=T/n
    table_prix=np.zeros((l,n+1),dtype=object)
    for i in range(l):
        table_prix[i,n]={reg:max(K-S_o*np.exp(arbre[i,n]*sigma_bar*np.sqrt(h)),0) for reg in range(len(List_sigma))}
    for j in range(1,n+1):
        for i in range(l):
            if i>=b*j and i<=b*(2*n-j):
                table_prix[i,n-j]={reg:0 for reg in range(len(List_sigma))}
                for etat in range(len(List_sigma)):
                    r=List_r[etat]
                    saut=Liste_saut[etat]
                    for k in range(len(List_sigma)):
                        table_prix[i,n-j][etat] += P[etat,k]*(table_prix[i-saut,n-j+1][k]*proba_umd[0,etat]+table_prix[i+saut,n-j+1][k]*proba_umd[1,etat]+table_prix[i,n-j+1][k]*proba_umd[2,etat])
                    table_prix[i,n-j][etat]=np.exp(-r*h)*table_prix[i,n-j][etat]
                    table_prix[i,n-j][etat]=max(table_prix[i,n-j][etat],K-S_o*np.exp(arbre[i,n-j]*sigma_bar*np.sqrt(h)))
        
    return table_prix

In [ ]:
C1=prix_put_americain(100,94,1000,2,0.2,1,L1,L2,L3,L4,P)
C2=prix_put_americain(100,96,1000,2,0.2,1,L1,L2,L3,L4,P)
C3=prix_put_americain(100,98,1000,2,0.2,1,L1,L2,L3,L4,P)
C4=prix_put_americain(100,100,1000,2,0.2,1,L1,L2,L3,L4,P)
C5=prix_put_americain(100,102,1000,2,0.2,1,L1,L2,L3,L4,P)
C6=prix_put_americain(100,104,1000,2,0.2,1,L1,L2,L3,L4,P)
C7=prix_put_americain(100,106,1000,2,0.2,1,L1,L2,L3,L4,P)

In [ ]:
C=[C1, C2, C3, C4, C5, C6, C7]

In [ ]:
Df2 = pd.DataFrame({'S0':[i for i in range(94,107,2)], 'Tree':[B[:,0][2000] for B in C]})
Df2